In [1]:
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
x_train = mnist.train.images
y_label = mnist.train.labels


X = tf.placeholder(tf.float32, shape=[None,784])
Y = tf.placeholder(tf.float32, shape=[None,10])

W1 = tf.get_variable('W1',shape=[784,500],initializer= tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.random_normal([500]))

layer1 = tf.nn.relu(tf.matmul(X,W1)+B1)

W2 = tf.get_variable('W2',shape=[500,200],initializer= tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.random_normal([200]))

layer4 = tf.nn.relu(tf.matmul(layer1, W2)+B2)

W5 = tf.get_variable('W5',shape=[200,10],initializer= tf.contrib.layers.xavier_initializer())
B5 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer4, W5)+B5
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
# cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.AdamOptimizer(0.008).minimize(cost)

is_correct = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


In [4]:
training_epoch = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost=0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c/total_batch
    print("Epoch :", epoch , "avg_cost : ",avg_cost)
    


Epoch : 0 avg_cost :  0.3806675636294214
Epoch : 1 avg_cost :  0.20119478121738554
Epoch : 2 avg_cost :  0.17870508572933338
Epoch : 3 avg_cost :  0.17515607616102166
Epoch : 4 avg_cost :  0.16226237564263019
Epoch : 5 avg_cost :  0.1647068961235611
Epoch : 6 avg_cost :  0.14669094404544342
Epoch : 7 avg_cost :  0.15791352897374467
Epoch : 8 avg_cost :  0.14141118026846514
Epoch : 9 avg_cost :  0.13562646836042397
Epoch : 10 avg_cost :  0.1371109957019374
Epoch : 11 avg_cost :  0.13161456843160782
Epoch : 12 avg_cost :  0.13045178445288913
Epoch : 13 avg_cost :  0.1303113047006032
Epoch : 14 avg_cost :  0.12143777720122173


In [5]:
x_test = mnist.test.images
y_test = mnist.test.labels

In [6]:
sess.run(accuracy, feed_dict={X:x_test, Y:y_test})

0.9584

# overfitting 줄이기


## regularization
### lambda * sigma weight(square)


## Dropout
### 학습할땐 몇개 자르고(dropout rate(keep probability)=0.x, 테스트할땐 모두 합쳐서 테스트( dropout rate=0)

In [3]:
#dropout 레이어 정하기

x_train = mnist.train.images
y_label = mnist.train.labels


X = tf.placeholder(tf.float32, shape=[None,784])
Y = tf.placeholder(tf.float32, shape=[None,10])

W1 = tf.get_variable('W1',shape=[784,500], initializer= tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.random_normal([500]))
layer1 = tf.nn.relu(tf.matmul(X,W1)+B1)

keep_prob = tf.placeholder(tf.float32) 
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob) #keep_prob는 스칼라값이면 accuracy를 측정할수가 없음
#일반적으로 0.5~0.8

W2 = tf.get_variable('W2',shape=[500,200],initializer= tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.random_normal([200]))

layer4 = tf.nn.relu(tf.matmul(layer1, W2)+B2)
layer4 = tf.nn.dropout(layer4, keep_prob=keep_prob)

W5 = tf.get_variable('W5',shape=[200,10],initializer= tf.contrib.layers.xavier_initializer())
B5 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer4, W5)+B5
logits = tf.nn.dropout(logits, keep_prob=keep_prob)
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
# cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

is_correct = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [4]:
training_epoch = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost=0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys, keep_prob:0.5})
        avg_cost += c/total_batch
    print("Epoch :", epoch , "avg_cost : ",avg_cost)
    


Epoch : 0 avg_cost :  1.4177329598773614
Epoch : 1 avg_cost :  1.273314251141114
Epoch : 2 avg_cost :  1.2468053307316516
Epoch : 3 avg_cost :  1.2505072659795944
Epoch : 4 avg_cost :  1.2029073084484445
Epoch : 5 avg_cost :  1.2312993214347143
Epoch : 6 avg_cost :  1.2447450902245254
Epoch : 7 avg_cost :  1.241155284318056
Epoch : 8 avg_cost :  1.2311057782173163
Epoch : 9 avg_cost :  1.2102483039552498
Epoch : 10 avg_cost :  1.2212697169997473
Epoch : 11 avg_cost :  1.2054664245518774
Epoch : 12 avg_cost :  1.2107407922094509
Epoch : 13 avg_cost :  1.220084397034211
Epoch : 14 avg_cost :  1.2079613567482337


In [5]:
x_test = mnist.test.images
y_test = mnist.test.labels

In [6]:
sess.run(accuracy, feed_dict={X:x_test, Y:y_test, keep_prob:1})

0.9341